## Optional pip installation (for Winddows)

In [2]:
%pip install matplotlib torch==1.8.0 torchvision==0.9.0 gluoncv decord

  Using cached gluoncv-0.10.5.post0-py2.py3-none-any.whl (1.3 MB)
  Using cached decord-0.6.0-py3-none-win_amd64.whl (24.7 MB)
  Using cached pyparsing-3.1.1-py3-none-any.whl (103 kB)
  Using cached cycler-0.12.1-py3-none-any.whl (8.3 kB)
  Using cached yacs-0.1.8-py3-none-any.whl (14 kB)
  Using cached opencv_python-4.9.0.80-cp37-abi3-win_amd64.whl (38.6 MB)
  Using cached autocfg-0.0.8-py3-none-any.whl (13 kB)
  Using cached portalocker-2.8.2-py3-none-any.whl (17 kB)
  Using cached requests-2.31.0-py3-none-any.whl (62 kB)
Note: you may need to restart the kernel to use updated packages.


  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
You should consider upgrading via the 'c:\Users\Owner\.pyenv\pyenv-win\versions\3.8.10\python.exe -m pip install --upgrade pip' command.


In [10]:
%pip uninstall Pillow

^C
Note: you may need to restart the kernel to use updated packages.


In [18]:
%pip install Pillow

Note: you may need to restart the kernel to use updated packages.


You should consider upgrading via the 'c:\Users\Owner\.pyenv\pyenv-win\versions\3.8.10\python.exe -m pip install --upgrade pip' command.


In [12]:
%matplotlib inline

# 1. Getting Started with Pre-trained I3D Models on Kinetcis400

`Kinetics400 <https://deepmind.com/research/open-source/kinetics>`_  is an action recognition dataset
of realistic action videos, collected from YouTube. With 306,245 short trimmed videos
from 400 action categories, it is one of the largest and most widely used dataset in the research
community for benchmarking state-of-the-art video action recognition models.

`I3D <https://arxiv.org/abs/1705.07750>`_ (Inflated 3D Networks) is a widely adopted 3D video
classification network. It uses 3D convolution to learn spatiotemporal information directly from videos.
I3D is proposed to improve `C3D <https://arxiv.org/abs/1412.0767>`_ (Convolutional 3D Networks) by inflating from 2D models.
We can not only reuse the 2D models' architecture (e.g., ResNet, Inception), but also bootstrap
the model weights from 2D pretrained models. In this manner, training 3D networks for video
classification is feasible and getting much better results.

In this tutorial, we will demonstrate how to load a pre-trained I3D model from `gluoncv-model-zoo`
and classify a video clip from the Internet or your local disk into one of the 400 action classes.

## Step by Step

We will try out a pre-trained I3D model on a single video clip.

First, please follow the `installation guide <../../index.html#installation>`__
to install ``PyTorch`` and ``GluonCV`` if you haven't done so yet.

## Simon's Fixes to Installation Instructions

1. Use python 3.8
2. Run `pip install torch==1.6.0 torchvision==0.7.0 gluoncv decord`
3. Run `pip uninstall Pillow`
4. Run `pip install Pillow==9.5.0`
5. (Optional) install Jupyter lab to run example notebook linked in tutorial `pip install jupyterlab`
6. Download the model config to download the pretrained model used in the tutorial (you will need to edit the config file path to where this file is stored on your system when running the code block which loads the model): https://raw.githubusercontent.com/dmlc/gluon-cv/master/scripts/action-recognition/configuration/resnet50_v1b_kinetics400.yaml
7. Run the notebook and check if class 0 (abseiling) is the final output.

In [7]:
import numpy as np
import decord
import torch

from gluoncv.torch.utils.model_utils import download
from gluoncv.torch.data.transforms.videotransforms import video_transforms, volume_transforms
from gluoncv.torch.engine.config import get_cfg_defaults
from gluoncv.torch.model_zoo import get_model

Then, we download a video and extract a 32-frame clip from it.



In [107]:
url = 'https://github.com/bryanyzhu/tiny-ucf101/raw/master/abseiling_k400.mp4'  # contains 250 frames
video_fname = download(url)
vr = decord.VideoReader(video_fname)
frame_id_list = [5, 6, 7, 8, 9]
video_data = vr.get_batch(frame_id_list).asnumpy()

Now we define transformations for the video clip.
This transformation function does four things:
(1) resize the shorter side of video clip to short_side_size,
(2) center crop the video clip to crop_size x crop_size,
(3) transpose the video clip to ``num_channels*num_frames*height*width``,
and (4) normalize it with mean and standard deviation calculated across all ImageNet images.



In [108]:
crop_size = 224
short_side_size = 256
transform_fn = video_transforms.Compose([video_transforms.Resize(short_side_size, interpolation='bilinear'),
                                         video_transforms.CenterCrop(size=(crop_size, crop_size)),
                                         volume_transforms.ClipToTensor(),
                                         video_transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])])


clip_input = transform_fn(video_data)
print('Video data is downloaded and preprocessed.')

Video data is downloaded and preprocessed.


Next, we load a pre-trained I3D model. Make sure to change the ``pretrained`` in the configuration file to True.



In [109]:
config_file = './i3d_resnet50_v1_kinetics400.yaml'
cfg = get_cfg_defaults()
cfg.merge_from_file(config_file)
model = get_model(cfg)
model.eval()
print('%s model is successfully loaded.' % cfg.CONFIG.MODEL.NAME)

i3d_resnet50_v1_kinetics400 model is successfully loaded.


Finally, we prepare the video clip and feed it to the model.



In [110]:
with torch.no_grad():
    pred = model(torch.unsqueeze(clip_input, dim=0)).numpy()

# Convert raw logits to probabilities using softmax
probs = torch.nn.functional.softmax(torch.tensor(pred), dim=1).numpy()

# Get the top predicted class and calculate confidence interval
top_class = np.argmax(probs)
confidence_interval = np.max(probs) - np.min(probs)

print(f'The input video clip is classified as class {top_class} with confidence interval {confidence_interval}')

The input video clip is classified as class 0 with confidence interval 0.7715625166893005


## Calculate confidence of frame windows
##### Adjust the 'N' value to set the step size

In [116]:
N = 2

url = 'https://github.com/bryanyzhu/tiny-ucf101/raw/master/abseiling_k400.mp4' 
video_fname = download(url)
vr = decord.VideoReader(video_fname)
config_file = './i3d_resnet50_v1_kinetics400.yaml'
cfg = get_cfg_defaults()
cfg.merge_from_file(config_file)
model = get_model(cfg)
model.eval()
for i in range(2 * N, len(vr) - (2 * N), 2): 
    frame_id_list = range(i - (2*N), i + (2*N) + 1, N)
    video_data = vr.get_batch(frame_id_list).asnumpy()
    crop_size = 224
    short_side_size = 256
    transform_fn = video_transforms.Compose([video_transforms.Resize(short_side_size, interpolation='bilinear'),
                                            video_transforms.CenterCrop(size=(crop_size, crop_size)),
                                            volume_transforms.ClipToTensor(),
                                            video_transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])])


    clip_input = transform_fn(video_data)
    with torch.no_grad():
        pred = model(torch.unsqueeze(clip_input, dim=0)).numpy()

    # Convert raw logits to probabilities using softmax
    probs = torch.nn.functional.softmax(torch.tensor(pred), dim=1).numpy()

    # Get the top predicted class and calculate confidence interval
    top_class = np.argmax(probs)
    confidence_interval = np.max(probs) - np.min(probs)

    print(f'The input video clip is classified as class {top_class} with confidence interval {confidence_interval} for frame window {i}')


The input video clip is classified as class 0 with confidence interval 0.9443979263305664 for frame window 4
The input video clip is classified as class 0 with confidence interval 0.9288216233253479 for frame window 6
The input video clip is classified as class 0 with confidence interval 0.9487528204917908 for frame window 8
The input video clip is classified as class 0 with confidence interval 0.8573697805404663 for frame window 10
The input video clip is classified as class 0 with confidence interval 0.8656697273254395 for frame window 12
The input video clip is classified as class 0 with confidence interval 0.9996094107627869 for frame window 14
The input video clip is classified as class 0 with confidence interval 0.9967054724693298 for frame window 16
The input video clip is classified as class 0 with confidence interval 0.9980602860450745 for frame window 18
The input video clip is classified as class 0 with confidence interval 0.9998185038566589 for frame window 20
The input vid

##### function to measure average confidence given window size 'N' (Variable # frames)

In [117]:
def frame_window_confidence(N, vr, model, true_class):
    sum_confidence = 0
    sum_class = 0
    for i in range(N, len(vr) - N):
        frame_id_list = range(i-N, i+N+1)
        video_data = vr.get_batch(frame_id_list).asnumpy()
        crop_size = 224
        short_side_size = 256
        transform_fn = video_transforms.Compose([video_transforms.Resize(short_side_size, interpolation='bilinear'),
                                                video_transforms.CenterCrop(size=(crop_size, crop_size)),
                                                volume_transforms.ClipToTensor(),
                                                video_transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])])
        clip_input = transform_fn(video_data)
        with torch.no_grad():
            pred = model(torch.unsqueeze(clip_input, dim=0)).numpy()
        probs = torch.nn.functional.softmax(torch.tensor(pred), dim=1).numpy()
        top_class = np.argmax(probs)
        confidence_interval = np.max(probs) - np.min(probs)
        if top_class == true_class: 
            sum_class += 1
            sum_confidence += confidence_interval
        print(f'Class: {top_class} \tConfidence: {confidence_interval} \tWindow:{i}')
    print(f'Average confidence level for window size {N} is {sum_confidence / (len(vr) - 2*N)}')
    print(f'Predicted top class with accuracy {sum_class / (len(vr) - 2*N)}')


In [118]:
url = 'https://github.com/bryanyzhu/tiny-ucf101/raw/master/abseiling_k400.mp4' 
video_fname = download(url)
vr = decord.VideoReader(video_fname)
config_file = './i3d_resnet50_v1_kinetics400.yaml'
cfg = get_cfg_defaults()
cfg.merge_from_file(config_file)
model = get_model(cfg)
model.eval()

frame_window_confidence(4, vr, model, 0)

Class: 0 	Confidence: 0.9948937892913818 	Window:4
Class: 0 	Confidence: 0.9884089231491089 	Window:5
Class: 0 	Confidence: 0.9704090356826782 	Window:6
Class: 0 	Confidence: 0.5901897549629211 	Window:7
Class: 0 	Confidence: 0.6856477856636047 	Window:8
Class: 0 	Confidence: 0.6187102198600769 	Window:9
Class: 0 	Confidence: 0.9852107167243958 	Window:10
Class: 0 	Confidence: 0.9958914518356323 	Window:11
Class: 0 	Confidence: 0.8777950406074524 	Window:12
Class: 0 	Confidence: 0.9682585597038269 	Window:13
Class: 0 	Confidence: 0.7068588137626648 	Window:14
Class: 0 	Confidence: 0.9096207618713379 	Window:15
Class: 0 	Confidence: 0.9992818236351013 	Window:16
Class: 0 	Confidence: 0.9621496796607971 	Window:17
Class: 0 	Confidence: 0.9934117197990417 	Window:18
Class: 0 	Confidence: 0.9939846396446228 	Window:19
Class: 0 	Confidence: 0.9591202735900879 	Window:20
Class: 0 	Confidence: 0.9972841739654541 	Window:21
Class: 0 	Confidence: 0.9996269941329956 	Window:22
Class: 0 	Confiden

We can see that our pre-trained model predicts this video clip
to be ``abseiling`` action with high confidence.



## Next Step

If you would like to dive deeper into finetuing SOTA video models on your datasets,
feel free to read the next `tutorial on finetuning <finetune_custom.html>`__.

